In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl

In [ ]:
cosmo = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_no_lambda = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    w0 = 0., wa = 0,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_no_lambda_v2 = ccl.Cosmology(
    Omega_c=0.94, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_w0wa = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    w0=-0.8, wa=-1.0,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)


In [ ]:
zmin = 0.01
zmax = 6
nzBins = 300

In [ ]:
def z_to_a(z: np.ndarray) -> np.ndarray:
    return 1/(1+z)

def a_to_z(a: np.ndarray) -> np.ndarray:
    return 1/(1-a)

In [ ]:
z_grid = np.linspace(zmin, zmax, nzBins)
z_widths = z_grid[1:] - z_grid[0:-1]
a_grid = z_to_a(z_grid)

In [ ]:
z_widths.shape

In [ ]:
j = 1./cosmo.h_over_h0(a_grid)

In [ ]:
def int_j(cosmo, z_vals):
    a_vals = z_to_a(z_vals)
    j = 1./cosmo.h_over_h0(a_vals)
    z_widths = z_vals[1:] - z_vals[0:-1]    
    return np.cumsum(j)[1:] * z_widths

In [ ]:
_ = plt.plot(z_grid[1:], int_j(cosmo, z_grid), label="LCDM")
_ = plt.plot(z_grid[1:], int_j(cosmo_w0wa, z_grid), label="w0wa")
_ = plt.plot(z_grid[1:], int_j(cosmo_no_lambda, z_grid), label='wo = 0')
_ = plt.legend()

In [ ]:
_ = plt.plot(z_grid[1:], int_j(cosmo_w0wa, z_grid)/int_j(cosmo, z_grid), label="LCDM")


In [ ]:
_ = plt.plot(z_grid, cosmo.comoving_radial_distance(a_grid), label='LCDM')
_ = plt.plot(z_grid, cosmo_w0wa.comoving_radial_distance(a_grid), label='w0wa')
_ = plt.plot(z_grid, cosmo_no_lambda.comoving_radial_distance(a_grid), label='wo = 0')
_ = plt.plot(z_grid, cosmo_no_lambda_v2.comoving_radial_distance(a_grid), label='omega_b = 0.94')

_ = plt.legend()


In [ ]:
_ = plt.plot(z_grid, cosmo.h_over_h0(a_grid), label='LCDM')
_ = plt.plot(z_grid, cosmo_w0wa.h_over_h0(a_grid), label='w0wa')
_ = plt.plot(z_grid, cosmo_no_lambda.h_over_h0(a_grid), label='wo = 0')
_ = plt.plot(z_grid, cosmo_no_lambda_v2.h_over_h0(a_grid), label='omega_b = 0.94')

_ = plt.legend()

In [ ]:
_ = plt.plot(z_grid, cosmo_w0wa.h_over_h0(a_grid)/cosmo.h_over_h0(a_grid), label='wowa / LCDM')
_ = plt.plot(z_grid, cosmo.h_over_h0(a_grid)/cosmo_no_lambda.h_over_h0(a_grid), label='LCDM / wo = 0')
_ = plt.legend()


In [ ]:
mesh = np.meshgrid(z_grid, z_grid)

In [ ]:
def generalized_sin(x: np.ndarray, K: float=0) -> np.ndarray:
    scaleK = np.sqrt(np.fabs(K))
    if K < 0:
        return np.sin(scaleK*x)/scaleK
    elif K > 0:
        return np.sinh(scaleK*x)/scaleK
    return x

def generalized_cos(x: np.ndarray, K: float=0) -> np.ndarray:
    scaleK = np.sqrt(np.fabs(K))
    if K < 0:
        return np.cos(scaleK*x)/scaleK
    elif K > 0:
        return np.cosh(scaleK*x)/scaleK
    return np.ones(len(x))

def generalized_cot(x: np.ndarray, K: float=0) -> np.ndarray:
    scaleK = np.sqrt(np.fabs(K))
    if K < 0:
        return scaleK/np.tan(scaleK*x)
    elif K > 0:
        return scaleK/np.tanh(scaleK*x)
    return np.ones(len(x))



In [ ]:
def ratio(cosmo, z_d: np.ndarray, z_s: np.ndarray) -> np.ndarray:
    a_d = z_to_a(z_d)
    a_s = z_to_a(z_s)
    chi_d = cosmo.comoving_radial_distance(a_d.ravel()).reshape(a_d.shape)
    chi_s = cosmo.comoving_radial_distance(a_s.ravel()).reshape(a_s.shape)
    chi_l = chi_s - chi_d
    rat = (1 + (chi_d/chi_l))/a_s
    return np.where(z_s-z_d>0.2, rat, 0)

def chiRatio(cosmo, z_d: np.ndarray, z_s: np.ndarray) -> np.ndarray:
    a_d = z_to_a(z_d)
    a_s = z_to_a(z_s)
    chi_d = cosmo.comoving_radial_distance(a_d.ravel()).reshape(a_d.shape)
    chi_s = cosmo.comoving_radial_distance(a_s.ravel()).reshape(a_s.shape)
    chi_l = chi_s - chi_d
    rat = chi_d/chi_s
    return np.where(z_s-z_d>0.2, rat, 0)

In [ ]:
z_d = mesh[0]
z_s = mesh[1]
a_d = z_to_a(z_d)
a_s = z_to_a(z_s)
chi_d = cosmo_no_lambda.comoving_radial_distance(a_d.ravel()).reshape(a_d.shape)
chi_s = cosmo_no_lambda.comoving_radial_distance(a_s.ravel()).reshape(a_s.shape)
chi_l = chi_s - chi_d
rat_chi = chi_d/chi_l

In [ ]:
_ = plt.plot(z_grid, cosmo_w0wa.comoving_radial_distance(a_grid)/cosmo.comoving_radial_distance(a_grid))
_ = plt.xlabel('z')
_ = plt.ylabel(r"$\chi_{wowa} / \chi_{\Lambda CDM}$")

In [ ]:
r = ratio(cosmo, mesh[0], mesh[1])
r1 = ratio(cosmo_no_lambda, mesh[0], mesh[1])
rw = ratio(cosmo_w0wa, mesh[0], mesh[1])

In [ ]:
chi_r = chiRatio(cosmo, mesh[0], mesh[1])
chi_r1 = chiRatio(cosmo_no_lambda, mesh[0], mesh[1])
chi_rw = chiRatio(cosmo_w0wa, mesh[0], mesh[1])

In [ ]:
_ = plt.imshow(chi_r, origin='lower', norm='log', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"$R_{ds} = \chi_{s}/\chi_{d}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$\Lambda CDM$')

In [ ]:
_ = plt.imshow(chi_r/chi_r1, origin='lower', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"$R_{\Lambda CDM} / R_{CDM}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$\Lambda CDM / CDM$')

In [ ]:
_ = plt.imshow(chi_rw/chi_r, origin='lower', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"$R_{w_{0}w_{a}} / R_{\Lambda CDM}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$w_{0}w_{a} / \Lambda CDM$')

In [ ]:
_ = plt.imshow(r, origin='lower', norm='log', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"$D_{\Delta T}/D_{D}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$\Lambda CDM$')

In [ ]:
_ = plt.imshow(r1, origin='lower', norm='log', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"$D_{\Delta T}/D_{D}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'CDM Only')

In [ ]:
_ = plt.imshow(rw, origin='lower', norm='log', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"$D_{\Delta T}/D_{D}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$w_{0}w_{a}$')

In [ ]:
_ = plt.imshow((r/r1).clip(0.,5.0), origin='lower', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"R $D_{\Delta T}/D_{D}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$\Lambda CDM$ / CDM Only')

In [ ]:
_ = plt.imshow((rw/r).clip(0.,5.0), origin='lower', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"R $D_{\Delta T}/D_{D}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$w_{0}w_{a} / \Lambda CDM$')